In [1]:
from skimage import io
import numpy as np
import os

ModuleNotFoundError: No module named 'skimage'

In [4]:
# filenames are structured with '_w1', '_w2', '_w3' to indicate channels.
directory = '../images/to_combine'
output_directory = '../images'
os.makedirs(output_directory)

In [28]:

# Create lists for different wavelength images
w1_images, w2_images, w3_images = [], [], []

# Populate the lists with corresponding image filenames
for filename in sorted(os.listdir(directory)):
    if ("_w1" in filename) and ('._helix' not in filename):
        w1_images.append(filename)
    elif ("_w2" in filename) and ('._helix' not in filename):
        w2_images.append(filename)
    elif ("_w3" in filename) and ('._helix' not in filename):
        w3_images.append(filename)

In [29]:
for w1, w2, w3 in zip(w1_images, w2_images, w3_images):
    # Read the images
    img_w1 = io.imread(os.path.join(directory, w1), as_gray=True)
    img_w2 = io.imread(os.path.join(directory, w2), as_gray=True)
    img_w3 = io.imread(os.path.join(directory, w3), as_gray=True)

    # Stack the images along the third axis to create an RGB image
    # Note: Ensure that the images are correctly scaled (0-1 range for floats, 0-255 for integers)
    img_w1 = img_w1 / img_w1.max()
    img_w2 = img_w2 / img_w2.max()
    img_w3 = img_w3 / img_w3.max()

    rgb_image = np.stack((img_w3, img_w2, img_w1), axis=-1)
    
    # Assume w1 is something like 'image1.tif'
    base_filename = os.path.splitext(w1)[0]  # This will remove the .tif and give 'image1'    
    output_filename = 'RGB_{}.tif'.format(base_filename)
    # Save or display the image
    io.imsave(os.path.join(output_directory, output_filename), (rgb_image * 255).astype(np.uint8))
    # io.imshow(rgb_image)  # Display the image